# Tutorial: REML estimation of variance components

Theo Meuwissen

## Sire model
The data **y** are affected by:
- a sire effect (10 independent sires), 
- a herd effect (2 herd effects)
- an independent residual effect (contains genetic effects of dam and Mendelian sampling + environmental effect)

The model: $\mathbf{y = Xb + Zu + e}$
- **b**: herd effect (2 herds)
- **u**: sire effect (10 sires)
- **e**: residual effect (for 100 offspring)

- heritability is $h^2 = 0.40$ 
- sire variance is $\sigma_s^2 = 0.10$
- residual variance is $\sigma_e^2 = 0.90$


## Simulate some sire model data
- 10 sires with 10 offspring each
- 2 herds: uneven offspringIDs in Herd1; even offspringIDs in Herd2
- note: since data are randomly sampled: repetition of results below will give different answer

In [15]:
using LinearAlgebra
s2=0.10
e2=0.90
q=10                       # number of sires
nd=10                      # daughters per sire
n=q*nd                     # total number of records
b=randn(2)
u=randn(q)*sqrt(s2)       # ~N(0,s2)
e=randn(n)*sqrt(e2)      # ~N(0,e2)
X=kron(ones(Int(n/2),1),I(2))    #design matrix for herd (kron=kronecker product)
Z=kron(I(q),ones(nd,1))   #design matrix for sire
y=X*b+Z*u+e                #the data


100-element Vector{Float64}:
 -3.2262439771102085
 -0.2177688288830595
 -3.827017463990323
 -1.2792352608621251
 -2.737852558391992
 -0.7277420972205069
 -2.1654699585908057
 -1.4147880205783288
 -2.490579920302945
 -1.150446633643876
 -2.487150159344688
  0.2802919595222342
 -1.8038714657700716
  ⋮
 -4.993811034587874
  0.3531092900475421
 -2.48854795349344
  1.2883891564009315
 -2.2029802357118777
  1.6799085370077984
 -3.161570119614587
 -0.36463698145418966
 -4.2294749096553135
 -0.6952472931789662
 -2.1232873012997238
  0.8087860951169439

# Estimation by  EM - REML
We use true values of variance components as starting values.
- slides 5 and 5 of estimation_of_variance_components.ppt
- note: the same BLUP solutions and PEV are obtained by the usual Mixed Model Equations incl. fixed effects
- note2: no A-inverse is needed here because sires are unrelated 




In [16]:
S=I(100)-X*inv(X'*X)*X'                  # the projection matrix S: S*y = residuals after fitting herd
lambda=e2/s2                             #sire model lambda = ratio of residual to sire variance
MMM=Z'*S*Z+I(10)*lambda                  #Mixed Model equation matrix (after absorption of fixed effects)
uhat=inv(MMM)*Z'*S*y                     #BLUP solution of sire effects
s2hat=(uhat'*uhat+tr(inv(MMM)*e2))/q     #update of sire variance
bhat=inv(X'*X)*(X'*y-X'*Z*uhat)           # from usual Mixed Model Equations (fixed effect part)
ehat=y-X*bhat-Z*uhat                      #estimate of residuals
e2hat=(ehat'*ehat+tr(inv(MMM)*Z'*Z)*e2)/n #update of error variance
[s2hat e2hat]                              #show results




1×2 Matrix{Float64}:
 0.110114  0.968879

## Estimation by Average Information - REML
Again use true values of variance components as starting values (note: iterative algorithms need starting values and good starting values help convergence (especially for AI-REML)).
- slide 8 of estimation_of_variance_components.ppt shows calculation of Score functions
- slide 9 shows calculation of second derivatives
- again: no A-inverse is needed here because sires are assumed unrelated
- slide 7 in Maximum_likelihood_estimation.ppt gives formula for update in 2nd order algorithms (average info)
- note: P is calculated directly below, but it is more efficient to calculate terms like d'*P*d for any d (but treating d as records), as d'*P*d = (d-Xh)'*inv(V)*(d-Xh) = d'*inv(R)*ehat, where h=estimate of fixed effects IF d were actual records, and ehat=estimates of residuals if d were records. The latter is only requires estimation of BLUP solutions given d as records, not the inversion of V.   



In [17]:
V=Z*Z'*s2+I(n)*e2                #(co)variance matrix of records
VI=inv(V)
P=VI-VI*X*inv(X'*VI*X)*X'*VI     #Projection matrix
Score=zeros(2)                   #Initialise Score vector of dimension 2 since there are 2 variance components
Score[1]=-0.5*(q/s2-tr(inv(MMM))*e2/(s2^2)-uhat'*uhat/(s2^2))
Score[2]=-0.5*((n-q)/e2-tr(inv(MMM))*lambda/e2-ehat'*ehat/(e2^2))
AI=zeros(2,2)                      # (2x2) Average Information Matrix of second derivatives
AI[1,1]=uhat'*Z'*P*Z*uhat/(s2^2)
AI[2,2]=ehat'*P*ehat/(e2^2)
AI[1,2]=ehat'*P*Z*uhat/(e2*s2)
AI[2,1]=AI[1,2]                     
update=-inv(AI)*Score            # =(theta - theta0)
theta=[s2; e2] + update


2-element Vector{Float64}:
 0.09081008934116064
 0.8247537664543426

# Notes:
- both AI-REML and EM-REML are iterative algorithms
- this implies that after updated estimates are obtained, BLUP solutions are recalculated and new updates are calculated until the changes in solutions become negligibly small (the algorithm converges)
- the AI algorithm may never converge. The EM algorithm is guaranteed to converge, but it may be slow. If the AI algorithm does not converge, one may try some iterations of the EM algorithm.
- also: we need (good) starting values for the variance components to start the algorithm